In [18]:
import openai
from dotenv import load_dotenv
import os
from typing import Dict, Any
from yandex_cloud_ml_sdk import YCloudML

load_dotenv()
FOLDER_ID = os.getenv("FOLDER_ID")
API_KEY = os.getenv("MISTRAL_API_KEY")

# Инициализируем модель
sdk = YCloudML(
    folder_id=FOLDER_ID, auth=API_KEY
)
model = sdk.models.completions("yandexgpt-lite", model_version="rc")
model = model.configure(temperature=0.2)

def generate_kpi_summary(kpi_data: Dict[str, Any], model=model) -> str:
    """
    Генерирует краткую текстовую сводку по KPI для менеджеров с помощью LLM.
    Аргументы:
        kpi_data: Словарь с данными KPI в заданном формате
        model: Модель (YandexGPT)
    Возвращает строку с текстовой сводкой (2-4 предложения)
    
    Пример вызова:
    summary = generate_kpi_summary(daily_strong_growth)
    """
    # Формируем промпт с четкими инструкциями
    prompt = f"""
Ты — опытный бизнес-аналитик в e-commerce. Подготовь краткую аналитическую сводку для руководства на основе следующих данных:

Период: {kpi_data['period']}
Сравниваемый период: {kpi_data['comparison_period']}
Метрики:
{format_metrics(kpi_data['metrics'])}

Инструкции:
1. Используй ТОЛЬКО предоставленные данные.
2. Объем: ровно 3 предложения.
3. Первое предложение — основной тренд по GMV (значение и % изменения).
4. Второе предложение — ключевые драйверы (если изменение >10%).
5. Третье предложение — общий вывод (без домыслов).
6. Стиль: профессиональный, без эмоций, только факты.
7. Формат чисел: 1.25 млн руб, 12.5%, округлять до 1 знака после запятой.
8. Не упоминай метрики с изменением менее 5%.

Пример правильного ответа:
"15 ноября GMV составил 1.6 млн руб (+25%). Рост обеспечен увеличением заказов на 20% и сессий на 18%. Показатели демонстрируют согласованный рост по всем ключевым метрикам."
    """
    
    try:
        response = model.run(
            [
                {"role": "system", "text": "Ты — ИИ-ассистент аналитика, который генерирует точные и краткие сводки."},
                {
                    "role": "user",
                    "text": prompt,
                },
            ]
        )
        
        # Постобработка ответа
        summary = response
        return validate_summary(summary, kpi_data)
    
    except Exception as e:
        return f"Ошибка генерации сводки: {str(e)}"

def format_metrics(metrics: Dict[str, Any]) -> str:
    # Форматирует метрики для промпта
    return "\n".join(
        f"- {name}: {value['value']} ({value['change']*100:+.1f}%)"
        for name, value in metrics.items()
    )

def validate_summary(summary: str, kpi_data: Dict[str, Any]) -> str:
    # Проверяет соответствие сводки исходным данным. Если обнаружены расхождения, возвращает более строгий шаблонный ответ.
    required_phrases = [
        str(kpi_data['period']),
        f"{kpi_data['metrics']['GMV']['change']*100:+.1f}%"
    ]
    
    if not all(phrase in summary for phrase in required_phrases):
        return generate_fallback_summary(kpi_data)
    
    return summary

def generate_fallback_summary(kpi_data: Dict[str, Any]) -> str:
    # Генерирует шаблонную сводку, если LLM ошибается
    gmv = kpi_data['metrics']['GMV']
    gmv_value = format_number(gmv['value'])
    gmv_change = gmv['change'] * 100
    
    main_metrics = []
    for name, metric in kpi_data['metrics'].items():
        if name != 'GMV' and abs(metric['change']) >= 0.1:  # >10%
            main_metrics.append(f"{name} на {metric['change']*100:+.1f}%")
    
    if gmv_change > 5:
        trend = "рост"
    elif gmv_change < -5:
        trend = "снижение"
    else:
        trend = "стабильность"
    
    drivers = ", ".join(main_metrics) if main_metrics else "изменений по ключевым метрикам"
    
    return (
        f"{kpi_data['period']} GMV составил {gmv_value} ({gmv_change:+.1f}%). "
        f"Основные изменения: {drivers}. "
        f"Наблюдается {trend} основных показателей."
    )

def format_number(value: float) -> str:
    # Форматирует большие числа для читаемости
    if value >= 1_000_000:
        return f"{value/1_000_000:.1f} млн руб"
    elif value >= 1_000:
        return f"{value/1_000:.1f} тыс. руб"
    return str(value)


Проверим работу алгоритма:

In [ ]:
if __name__ == "__main__":
    daily_strong_growth = {
        "period": "2023-11-15",
        "comparison_period": "2023-11-14",
        "metrics": {
            "GMV": {"value": 1625000, "change": 0.25},
            "Заказы": {"value": 2400, "change": 0.20},
            "Средний чек (AOV)": {"value": 677.08, "change": 0.042},
            "Сессии": {"value": 60000, "change": 0.18},
            "Конверсия (CR)": {"value": 0.04, "change": 0.017}
        }
    }
    
    weekly_decline_traffic_anomaly = {
        "period": "Неделя 47, 2023",
        "comparison_period": "Неделя 46, 2023",
        "metrics": {
            "GMV": {"value": 7332000, "change": -0.06},
            "Заказы": {"value": 12420, "change": -0.08},
            "Средний чек (AOV)": {"value": 590.34, "change": 0.022},
            "Сессии": {"value": 472500, "change": 0.05},
            "Конверсия (CR)": {"value": 0.0263, "change": -0.124}
        }
    }
    
    daily_mixed_gmv_up_orders_down = {
        "period": "2023-11-17",
        "comparison_period": "2023-11-16",
        "metrics": {
            "GMV": {"value": 1060000, "change": 0.082},
            "Заказы": {"value": 1710, "change": -0.05},
            "Средний чек (AOV)": {"value": 619.88, "change": 0.139},
            "Сессии": {"value": 49000, "change": 0.021},
            "Конверсия (CR)": {"value": 0.0349, "change": -0.069}
        }
    }
    
    # Демонстрация работы
    for data in [daily_strong_growth, weekly_decline_traffic_anomaly, daily_mixed_gmv_up_orders_down]:
        print("\n" + "="*50)
        print("Исходные данные:")
        print(data)
        print("\nСводка:")
        print(generate_kpi_summary(data))
        print("="*50)


Исходные данные:
{'period': '2023-11-15', 'comparison_period': '2023-11-14', 'metrics': {'GMV': {'value': 1625000, 'change': 0.25}, 'Заказы': {'value': 2400, 'change': 0.2}, 'Средний чек (AOV)': {'value': 677.08, 'change': 0.042}, 'Сессии': {'value': 60000, 'change': 0.18}, 'Конверсия (CR)': {'value': 0.04, 'change': 0.017}}}

Сводка:
Ошибка генерации сводки: no explicit authorization data was passed and no authorization data was found at environment

Исходные данные:
{'period': 'Неделя 47, 2023', 'comparison_period': 'Неделя 46, 2023', 'metrics': {'GMV': {'value': 7332000, 'change': -0.06}, 'Заказы': {'value': 12420, 'change': -0.08}, 'Средний чек (AOV)': {'value': 590.34, 'change': 0.022}, 'Сессии': {'value': 472500, 'change': 0.05}, 'Конверсия (CR)': {'value': 0.0263, 'change': -0.124}}}

Сводка:
Ошибка генерации сводки: no explicit authorization data was passed and no authorization data was found at environment

Исходные данные:
{'period': '2023-11-17', 'comparison_period': '2023-

Всё работает отлично!

### Почему именно YandexGPT-lite?

1. <b>Оптимальное соотношение цена/качество: </b>   
    YandexGPT-lite — это облегчённая версия флагманской модели Яндекса, специально разработанная для задач с ограниченным бюджетом.   
    Для генерации кратких аналитических сводок (3 предложения) её возможностей более чем достаточно, при этом стоимость запроса значительно ниже, чем у yandexgpt-pro.

2. <b>Поддержка русского языка:   </b>   
    Модель превосходно работает с русскоязычными бизнес-терминами и профессиональной лексикой, что критично для e-commerce аналитики (GMV, конверсия, средний чек и т.д.).

3. <b>Контролируемая детерминированность:   </b>   
    Установка temperature=0.2 позволяет получать более предсказуемые и фактологически точные формулировки, что важно для отчётов руководству.

4. <b>Интеграция с Yandex Cloud:   </b>   
    Готовый SDK и простая аутентификация через API-ключ ускоряют развёртывание решения. Для корпоративных клиентов Яндекса есть дополнительные преимущества в SLA.

5. <b>Локальная специфика:   </b>   
    Модель лучше понимает российские реалии (например, форматы дат "15 ноября" вместо "November 15") и метрики в рублях.

### Возможные улучшения

- Fine-tuning на исторических отчётах компании для соблюдения корпоративного стиля

- Добавление RAG (Retrieval-Augmented Generation) с базой знаний о продукте

- Поддержка голосового ввода

- Переход на YandexGPT-pro